<a href="https://colab.research.google.com/github/nikodemadamski/FYP/blob/main/FYP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Start

## importing

importing the data sets directly from github along side multiple handy imports i will to use in the future.

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
%matplotlib inline
import requests
import io
import re

In [2]:
url = 'https://raw.githubusercontent.com/nikodemadamski/FYP/main/celebrities%20dataset.csv'
url2 = 'https://raw.githubusercontent.com/nikodemadamski/FYP/main/tiktok_1(1).csv'
download = requests.get(url).content
download2 = requests.get(url2).content


df = pd.read_csv(io.StringIO(download.decode('utf-8')))
df2 = pd.read_csv(io.StringIO(download2.decode('utf-8')))

### testing

In [3]:
print(df.head(10))

     Month  Charli D'Amelio web  Charli D'Amelio yt Charli D'Amelio image  \
0  2016-01                  5.0                 5.0                     3   
1  2016-02                  4.0                 5.0                     2   
2  2016-03                  4.0                 4.0                     2   
3  2016-04                  4.0                 4.0                     1   
4  2016-05                  3.0                 4.0                     1   
5  2016-06                  3.0                 3.0                     1   
6  2016-07                  3.0                 4.0                     1   
7  2016-08                  3.0                 4.0                     1   
8  2016-09                  3.0                 3.0                     2   
9  2016-10                  3.0                 4.0                     1   

   Charli D'Amelio news  Charli D'Amelio Total  Jules LeBlanc web  \
0                   1.0                   3.50                8.0   
1             

In [4]:
print(df2.head(100))

               Name  Following Followers  Likes  Views      V_Date V_Likes  \
0   charlidamelio         1267    136.7M  10.6B   1.4M     3h ago   342.1K   
1   charlidamelio         1267    136.7M  10.6B   4.4M     2d ago   799.3K   
2   charlidamelio         1267    136.7M  10.6B    27M     4d ago     5.8M   
3   charlidamelio         1267    136.7M  10.6B   9.8M     4d ago     1.5M   
4   charlidamelio         1267    136.7M  10.6B   6.5M     4d ago   828.5K   
..              ...        ...       ...    ...    ...         ...     ...   
95  charlidamelio         1267    136.7M  10.6B  18.8M  2021-12-10    2.4M   
96  charlidamelio         1267    136.7M  10.6B  16.2M  2021-12-10    2.1M   
97  charlidamelio         1267    136.7M  10.6B    18M   2021-12-9    1.4M   
98  charlidamelio         1267    136.7M  10.6B    19M   2021-12-9    2.7M   
99  charlidamelio         1267    136.7M  10.6B  17.2M   2021-12-9      2M   

   V_Comments  
0         19K  
1       55.6K  
2       83.1K  

In [5]:
df2.dtypes

Name          object
Following      int64
Followers     object
Likes         object
Views         object
V_Date        object
V_Likes       object
V_Comments    object
dtype: object

## Eda

I have to now change data set 2 , this will be in 2 ways, 1 will eliminate all data with V_date from after 2021, Second will average the videos from daily videos to monthly videos averages to be able to correlate with the data from the first data set

### 1

 eliminating all data after 2021
 [Reference](https://www.statology.org/pandas-sum-column-with-condition/)

In [6]:
df2 = df2[df2["V_Date"].str.contains("ago")==False]

In [7]:
df2 = df2[df2['V_Date'].between('2016-01-01', '2021-31-01')]

### 1.5
 Changing B into Billions, M into million and K into Thousands
 And Changing notation on values that are too large e.g 10.6B from Float to int as none of the large values have an exact amount so they are rounded to the nearest Billion,Million or thousand
  [Reference](https://stackoverflow.com/questions/39684548/convert-the-string-2-90k-to-2900-or-5-2m-to-5200000-in-pandas-dataframe)
  [Reference2](https://stackoverflow.com/questions/658763/how-to-suppress-scientific-notation-when-printing-float-values)

In [8]:
df2.Views = (df2.Views.replace(r'[KMB]+$', '', regex=True).astype(float) * \
                   df2.Views.str.extract(r'[\d\.]+([KMB]+)', expand=False)
                   .fillna(1)
                   .replace(['K','M','B'], [10**3, 10**6,10**9]).astype(int))

In [9]:
df2.V_Likes = (df2.V_Likes.replace(r'[KMB]+$', '', regex=True).astype(float) * \
                   df2.V_Likes.str.extract(r'[\d\.]+([KMB]+)', expand=False)
                   .fillna(1)
                   .replace(['K','M','B'], [10**3, 10**6,10**9]).astype(int))

In [10]:
df2.V_Comments = (df2.V_Comments.replace(r'[KMB]+$', '', regex=True).astype(float) * \
                   df2.V_Comments.str.extract(r'[\d\.]+([KMB]+)', expand=False)
                   .fillna(1)
                   .replace(['K','M','B'], [10**3, 10**6,10**9]).astype(int))

In [11]:
df2.Likes = (df2.Likes.replace(r'[KMB]+$', '', regex=True).astype(float) * \
                   df2.Likes.str.extract(r'[\d\.]+([KMB]+)', expand=False)
                   .fillna(1)
                   .replace(['K','M','B'], [10**3, 10**6,10**9]).astype(int))

In [12]:
df2.Followers = (df2.Followers.replace(r'[KMB]+$', '', regex=True).astype(float) * \
                   df2.Followers.str.extract(r'[\d\.]+([KMB]+)', expand=False)
                   .fillna(1)
                   .replace(['K','M','B'], [10**3, 10**6,10**9]).astype(int))

In [13]:
print(df2.head(10))

               Name  Following    Followers         Likes       Views  \
70  charlidamelio         1267  136700000.0  1.060000e+10  16800000.0   
71  charlidamelio         1267  136700000.0  1.060000e+10  15200000.0   
72  charlidamelio         1267  136700000.0  1.060000e+10  12000000.0   
73  charlidamelio         1267  136700000.0  1.060000e+10  13400000.0   
74  charlidamelio         1267  136700000.0  1.060000e+10  21400000.0   
75  charlidamelio         1267  136700000.0  1.060000e+10  15000000.0   
76  charlidamelio         1267  136700000.0  1.060000e+10  13100000.0   
77  charlidamelio         1267  136700000.0  1.060000e+10  20600000.0   
78  charlidamelio         1267  136700000.0  1.060000e+10  13100000.0   
79  charlidamelio         1267  136700000.0  1.060000e+10  25700000.0   

        V_Date    V_Likes  V_Comments  
70  2021-12-31  2100000.0     43600.0  
71  2021-12-29  1000000.0     24400.0  
72  2021-12-28   693300.0     15800.0  
73  2021-12-28   859300.0     22200.

In [14]:
# repl_dict = {'[kK]': '*1e3', '[mM]': '*1e6', '[bB]': '*1e9', }
# df2['Views'].replace(repl_dict, regex=True).map(pd.eval)

In [15]:
tList = ['Following','Followers','Likes','Views','V_Likes','V_Comments']
for x in tList:
  df2[x]=df2[x].astype(int)

In [16]:
print(df2.dtypes)

Name          object
Following      int64
Followers      int64
Likes          int64
Views          int64
V_Date        object
V_Likes        int64
V_Comments     int64
dtype: object


In [29]:
print(df2.head(5000))

                  Name  Following  Followers        Likes     Views  \
70     charlidamelio         1267  136700000  10600000000  16800000   
71     charlidamelio         1267  136700000  10600000000  15200000   
72     charlidamelio         1267  136700000  10600000000  12000000   
73     charlidamelio         1267  136700000  10600000000  13400000   
74     charlidamelio         1267  136700000  10600000000  21400000   
...                ...        ...        ...          ...       ...   
5829  jacobsartorius         2532   23800000   1600000000    443500   
5830  jacobsartorius         2532   23800000   1600000000   1500000   
5831  jacobsartorius         2532   23800000   1600000000   2100000   
5832  jacobsartorius         2532   23800000   1600000000    460900   
5833  jacobsartorius         2532   23800000   1600000000   1500000   

         V_Date  V_Likes  V_Comments  
70   2021-12-31  2100000       43600  
71   2021-12-29  1000000       24400  
72   2021-12-28   693300      

### 2

averaging into monthly videos

first lets change the V_date from a string into a valid time

In [18]:
df2["V_Date"] = pd.to_datetime(df2["V_Date"])

next lets try to group by time
  [Reference](https://stackoverflow.com/questions/65471540/get-monthly-average-in-pandas)

In [19]:
tList = ['Following','Followers','Likes','Views','V_Likes','V_Comments']
for x in tList:
  df2[x]=df2[x].astype(int)

In [20]:
df3 = df2.groupby(pd.PeriodIndex(df2['V_Date'], freq="M"))['Views','V_Likes','V_Comments'].mean()
print(df3)

                Views       V_Likes    V_Comments
V_Date                                           
2016-01  5.484648e+05  4.999160e+05   2965.936585
2016-02  4.057512e+05  3.896661e+05   2134.005405
2016-03  4.627162e+05  4.498662e+05   2245.897778
2016-04  5.292238e+05  5.059840e+05   3287.063415
2016-05  1.274229e+06  5.173310e+05   5456.022901
...               ...           ...           ...
2021-02  1.399370e+07  1.812822e+06  31609.176245
2021-03  1.487278e+07  1.908479e+06  28983.425606
2021-10  9.709744e+06  1.224237e+06  15677.537931
2021-11  8.835518e+06  1.089247e+06  10806.786571
2021-12  8.275140e+06  9.479924e+05  11390.745140

[64 rows x 3 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [21]:
print(df3.dtypes)

Views         float64
V_Likes       float64
V_Comments    float64
dtype: object


In [22]:
tList = ['Views','V_Likes','V_Comments']
for x in tList:
  df3[x]=df3[x].astype(int)

In [28]:
print(df3)

            Views  V_Likes  V_Comments
V_Date                                
2016-01    548464   499916        2965
2016-02    405751   389666        2134
2016-03    462716   449866        2245
2016-04    529223   505983        3287
2016-05   1274228   517331        5456
...           ...      ...         ...
2021-02  13993701  1812821       31609
2021-03  14872775  1908478       28983
2021-10   9709744  1224236       15677
2021-11   8835518  1089247       10806
2021-12   8275139   947992       11390

[64 rows x 3 columns]


##EDA2

divide the df into a 2nd dataset whcih will combine all totals into people who do not have tiktok which will be my control data and people who do have it,
further analysis will continue by dividing the celebrities up into more groups for a more in debth analysis

  [Reference](https://stackoverflow.com/questions/19071199/drop-columns-whose-name-contains-a-specific-string-from-pandas-dataframe)


In [24]:
n_list = ['web', 'yt', 'image', 'news']
for x in n_list:
  df_total = df[df.columns.drop(list(df.filter(regex=x)))]

In [25]:
print(df_total.head(72))

      Month  Charli D'Amelio web  Charli D'Amelio yt Charli D'Amelio image  \
0   2016-01                  5.0                 5.0                     3   
1   2016-02                  4.0                 5.0                     2   
2   2016-03                  4.0                 4.0                     2   
3   2016-04                  4.0                 4.0                     1   
4   2016-05                  3.0                 4.0                     1   
..      ...                  ...                 ...                   ...   
67  2021-08                 41.0                25.0                    48   
68  2021-09                 43.0                28.0                    41   
69  2021-10                 29.0                17.0                    32   
70  2021-11                 29.0                16.0                    34   
71  2021-12                 31.0                16.0                    36   

    Charli D'Amelio Total  Jules LeBlanc web  Jules LeBlanc yt 